# 猜拳游戏


## 游戏规则

>有剪刀、石头、布三个手势。二人同时用手做出相应形状而出，输赢判断规则为：剪刀赢布，布赢石头，石头赢剪刀。

基于skids开发猜拳游戏：人机对战。

玩家输入：skids上的4个按键：S1-石头/右移，S2-剪刀/下移动，S3-布/左移，S4确认/上移

游戏结果：LCD屏幕输出文字或图案


## 设计思路

游戏设计上可拆分为几个部分：
* 显示-图案或文字的显示，用于输出游戏结果或提醒；
* 输入-按键输入用户的选择；
* 逻辑-判断游戏的胜负

实现上我们计划由简至繁来完成，先完成一个具备游戏功能的框架，然后在进行逐步完善。

# 简易版猜拳游戏

为了实现上的简单，我们先不支持显示功能，只使用终端输出进行游戏设计。终端输出实际上就是软件调试中最常用的方法，最经典的hello world就是在终端显示的。

简易版的思路是：

* 用输出代替屏幕
* 随机数对应石头、剪刀、布
* 实现按键输入对应石头、剪刀、布
* 实现胜负逻辑判断

## 输出代替屏幕


输入可以通过blockly输入，也可以直接在命令行输入。blockly中选择"文本"->"输出":

![](./picture/L001_hello_world.png)

在uPyCraft工具中找到PC识别到Skids的串口进行选择，正常情况下uPyCraft左侧device中会有文件`boot.py`被识别到。把blockly中生成的代码复制到boot.py文件中，下载并执行。

![](./picture/L001_uPyCraft.png)

这里也可以是命令行直接输入，在上图最下方">>>"位置输入：

In [1]:
print('hello world')

hello world


尝试中文输出：大家好，我是skids。

In [2]:
print('大家好，我是skids')

大家好，我是skids


在这里我们会发现，命令行中段不能输入中文，但是可以在文件编辑中输入，并从终端打印出来。这是因为命令输入中对中文没有支持。

注意：这里要用到uPyCraft

同样的方法，我们实现游戏开机内容打印：

````
猜拳游戏

S1-石头  S2-剪刀
S3-布   S4确认
```

//TODO



![](./picture/L001_start_no_lcd.png)

## 随机数

### 生成随机数

Skids代表的另外一方要模拟对手完成游戏，那么就需要能够随机的输出：石头、剪刀、布。比较简单的想法是用随机出现的1，2，3代表。我们使用`数学`中的`随机数`模块来产生随机数，并用`输出`进行打印。

![](./picture/L001_random.png)

In [ ]:
import random

print('猜拳游戏')
print('S1-石头 S2-剪刀')
print('S3-布   S4-确认')
print(random.randint(1, 3))

这里出现了`import random`语句，其功能是**导入**random模块，以便可以使用random的一些功能。我们可以多执行几次来验证是否能随机的输出1，2，3。方法是在uPyCraft中device下的boo.py上右键选择`run`完成执行。

当然为了更方便的测试，我们会想到能不能让最后的语句`print(random.randint(1, 3))`不断的执行？

为实现这个目的，可使用循环执行的功能。这个功能在逻辑中可找到，`逻辑->重复 满足条件`，然后把上面的随机数输出放到重复里面。当然我们希望的是程序能不停的输出随机数，我们来简单的判断结果是否是随机的。

结果...有点悲剧，我们的uPycraft死机了。原因是输出的太快了。如何解决这个问题呢？

是不是可以等一会儿执行一次，例如1秒钟执行一次呢？这个方法是正确的，在程序设计尤其是单片机设计中延时是非常常见的。Blockly中的延时也是在逻辑中的。

死机问题怎么解决？可能需要使用任务管理器关掉uPycraft，然后重新启动此软件。重新打开后程序修改程序：
![](./picture/L001_random_test.png)

In [1]:
import random
import utime

# 启动输出语句省略

while True:
  print(random.randint(1, 3))
  utime.sleep_ms(1000);

ModuleNotFoundError: No module named 'utime'

这里导入了新的库`utime`这个是与时间相关的。并且出现了`while True:`，正是这个语句完成了重复循环功能，其执行结果是条件为True就一直执行后面的2条语句。这里我们发现后面要执行的两条语句有缩进，并且True后面有个`:`，这是python语言的语法规则，在循环语句后面需要加冒号，需要循环执行的部分需要缩进。程序执行的时候就根据缩进和冒号来识别不同功能的，还有utime和random后面的`.`有指向和选择的含义，如`utime.sleep_ms`表示调用utime.py文件中的`sleepms`函数。

### 根据随机数输出石头剪刀布

上面能不断产生随机树并且打印输出了，但是打印输出的是数字，如何把这些数字和`石头`，`剪刀`，`布`对用起来呢？我们可以先描述这样一段话：

* 如果随机数是1输出"石头" 
* 如果随机数是2输出"剪刀" 
* 如果随机数是3输出"布" 

这里可对上面的话进行逻辑拆分为：“如果” +  “随机数是1”条件成立 + 那么执行“输出"石头" ”

最后的：输出"石头"，我们上面已经学习过了。

`如果`和`条件成立`我们同样可以在`逻辑`模块组中找到对应的模块，分别是：`逻辑->如果+执行`和`逻辑->相等`

从模块到语句过程如下:
![](./picture/L001_if_study.png)
图片中的`1`是从`数学`中的`0`改过来的。

如上，自己添加输出积木完成`石头输出`。

In [4]:
if random.randint(1, 3) == 1:
  print('石头')

石头


这里的`if`和上面的`while`类似都属于python语言的关键字，当然在其他语言中也有这两个关键字。if与while不同的是，if是判断，while是循环。if后面判断的是一个逻辑结果为真还是为假，为真则执行if后面缩进部分的代码。同样if语句后面需要`:`冒号。

当我们尝试继续进行比较时发现需要再次调用`random.randin`，而此时的值又是一个新的随机值。我们是否能把`random.randin`的值保存起来，用这个值和1，2，3进行比较呢？
答案是肯定的，程序中这种可以动态存储一个数值的叫做变量（可以变化的量），我们可以在blockly中找到`变量->赋值为`模块，把这个模块和随机数产生模块进行拼组就可以把随机数的结果赋值给变量的，每个变量有需要有一个唯一的名字便于使用时进行区分，就像每个学生都有的一个学号一样。这里我们给变量命名为value

![](./picture/L001_variable.png)

对应代码为：

In [5]:
value = random.randint(1, 3)

定义过之后就可以在`变量`组中找到value并拖放使用了。

请尝试用value变量完成判断并打印`石头`。这里实际上有一个原则，那就是变量需要先定义后使用。

对应代码为：

In [ ]:
value = random.randint(1, 3)
if value == 1:
  print('石头')

### 尝试完成剪刀和布的判断并打印

翻转完成，可把这部分放入`重复`里进行循环执行。

In [6]:
if value == 2:
  print('剪刀')
if value == 3:
  print('布')

剪刀


完整代码：
``` python
import random
import utime

# 启动输出语句省略

while True:
  value = random.randint(1, 3)
  if value == 1:
    print('石头')
  if value == 2:
    print('剪刀')
  if value == 3:
    print('布')
  utime.sleep_ms(1000);
```

## 按键输入

### 按键原理图

按键是我们可以和程序进行交互的一种接口，就像是灯泡的开关一样，打开就亮，关闭就灭。按键传给程序的正是这个开关的变化。为了能够使用开关，我们需要先查看一下电路原理图，好弄明白开关与灯泡的关系。

![](./picture/L001_key_sch.png)

![](./PICTURE/L001_key_sch2.png)

### 按键创建与识别

上面原理图可以看到按键用到的引脚号分别是I34,35,36,39。并且引脚号与按键对应关系为

* S1 = KEY1 = I36
* S2 = KEY2 = I39
* S3 = KEY3 = I34
* S4 = KEY4 = I35

要想知道按键是否被按下需要用到Blockly中`Skdis`中`引脚`块组中的：
* 创建PIN对象
* 获取引脚电平

因为电路原理图中已经又从按键接了电阻到VCC，所以不需要上下拉。现在我们创建1个按键并把结果打印出来。
![](./picture/L001_key1_read.png)

In [ ]:
from machine import Pin
import utime


S1 = Pin(36,Pin.IN);
key_value = S1.value();

print(key_value)

对应的代码如上，这里出现了`from machine import Pin`代码，这是从`machine`文件或文件夹中`Pin`文件或类。上面的代码要想检测到按键的是否变化必须重复执行才可以。请尝试使用`逻辑`中的重复执行对上述慕课进行修改，实现按键状态的打印，注意需要做延时，不然仍会想随机数中那样不断打印的。完成后代码如下：

In [ ]:
from machine import Pin
import utime


S1 = Pin(36,Pin.IN);
while True:
  key_value = S1.value();

  print(key_value)
  utime.sleep_ms(1000);

### 按键结果判断

如果按键被按下那么可以认为，输入了石头、剪刀、布中的一个。这里把按键把结果打印出来，同时如果按键状态为按下，即输入结果为零则打印`石头`。根据上面的代码测试，我们可以知道按下按下时key_value结果为0，没有按时结果为1。

![](./picture/L001_key1_read_and_check.png)

In [7]:
from machine import Pin
import utime


S1 = Pin(36,Pin.IN);
while True:
  key_value = S1.value();

  print(key_value)
  utime.sleep_ms(1000);
  if key_value == 0:
    print('石头')


ModuleNotFoundError: No module named 'machine'

### 尝试识别其他按键并打印相应结果

几个步骤：
1. 创建按键
2. 循环中增加其他按键读入和对应的按键结果变量
3. 判断按键结果为0时打印输出‘石头’，‘剪刀’，‘布’

**参考：**

![](./picture/L001_3key_read_and_check.png)


### 用数字代替按键结果


打印处输入的是什么不是最终目的，最终目的是为了知道按键结果是什么然后与随机数进行对比的。那么按键结果要么是S1~S4中的一个，要么就是没有按按键，我们可以用数字1，2，3，4对应4个按键，0对应没有按键。这样我们也需要一个新的变量进行存储当前的按键值，只需要在进行输出的时候把按键的编号赋值给一个新的变量即可。

![](./picture/L001_key_read_and_check2.png)
这里的0，1，2，3，4在程序执行过程中是不会发生变化的，程序中叫做常量，与会变化的变量相对应。代码段

``` python
from machine import Pin
import utime

S1 = Pin(36,Pin.IN);
S2 = Pin(39,Pin.IN);
S3 = Pin(34,Pin.IN);
while True:
  key_value = 0
  key_tmp = S1.value();

  if key_tmp == 0:
    print('石头')
    key_value = 1
  # S2,S3省略

  print(key_value)
  utime.sleep_ms(1000);

```

## 随机数与按键同时打印

上面我们分别完成了随机数的产生和按键的获取，并可以把相应结果赋值给变量或打印输出。那么现在我们来把两部分内容进行合并，并在有按键按下时打印按键和随机数的值。

整合时最简单的方法是在有按键输入时产生一个随机值进行打印。
![](./picture/L001_key_read_and_check_and_random.png)

## 胜负判断逻辑

胜负分三种结果：胜、负、平。

每个人的输入只有三种情况，即3个值：1，2，3。那么最简单的我们可以枚举各种情况：

|人/按键|机/随机值|比赛结果|
|--|--|--|
|1|1|平|
|1|2|负|
|1|3|胜|
|2|1|胜|
|2|2|平|
|2|3|负|
|3|1|负|
|3|2|胜|
|3|3|平|


### 条件语句判胜负

我们只需要判断按键结果和随机数的值是什么然后根据上面的表打印结果即可。与之前用的条件语句不同的是这里的条件需要比较2组值的结果。这里需要用到逻辑运算中的与/且。如上表第一行可描述为：按键输入等于1且随机数也等于1时为平，这里我们使用if关键字写一行伪代码：

if 按键==1 且 随机数==1
    输出：平

这里的且就是blockly中`逻辑`组中的`且`

组合结果如下：
![](./picture/L001_game_result_1.png)


用同样的方法完成其他情况的判定
//todo

## 完成

至此一个十分简易的猜拳游戏机已经完成，但是里面还有不少问题：
1. 没有屏幕，智能通过观察终端结果进行交互
2. 按键不灵敏，这个可以修改延时时间进行改善，但是会带来一个问题，按键按下时会不断输出比赛结果，相当于按下按下后就不断循环比赛
